# 🅿️ Parking Spot Detection using GeoAI

This notebook demonstrates how to use the `geoai-py` library for detecting parking spots in high-resolution aerial imagery. The workflow includes installing dependencies, loading the raster image, detecting cars, vectorizing the results, and visualizing them on an interactive map.

---

## 1. Install Required Libraries

Install the `geoai-py` package which includes tools for geospatial AI, computer vision, and raster/vector processing.



In [1]:
%pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.3/90.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.1/568.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/

#2. Import Required Libraries

In [2]:
import geoai

#3. Load and Download Raster Image
We will use a parking lot image hosted on Hugging Face.

In [4]:
raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/parking_spots.tif"
)
raster_path = geoai.download_file(raster_url)

# 4. Visualize Raster Data
Visualize the parking lot image to understand its structure before processing.



In [5]:
geoai.view_raster(raster_url)

# 5. Initialize the Car Detector
GeoAI includes a built-in deep learning-based car detector.

In [6]:
detector = geoai.CarDetector()

Model path not specified, downloading from Hugging Face...


car_detection_usa.pth:   0%|          | 0.00/176M [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--giswqs--geoai/snapshots/6698907683cad47f1b85bf1ef4698ea086ba51a0/car_detection_usa.pth
Model loaded successfully


# 6. Extract parking Spots

Detect cars from the raster and generate a mask image representing detected objects. Extract parking spots from the image using the model and save the output image.

In [7]:
mask_path = detector.generate_masks(
    raster_path=raster_path,
    output_path="parking_masks.tif",
    confidence_threshold=0.5,
    mask_threshold=0.5,
    overlap=0.25,
    chip_size=(256, 256),
    min_object_area=10,
    # max_object_area=5000,
)

Dataset initialized with 8 rows and 14 columns of chips
Image dimensions: 2626 x 1479 pixels
Chip size: 256 x 256 pixels
Overlap: 25.0% (stride_x=192, stride_y=192)
CRS: EPSG:3857
Processing raster with 28 batches


100%|██████████| 28/28 [12:04<00:00, 25.87s/it]

Masks with confidence values saved to parking_masks.tif


#7. Convert Masks to Vector Polygons (GeoJSON)
Convert the predicted masks into polygon features and save them as a GeoJSON.

In [8]:
gdf = detector.vectorize_masks(
    masks_path="parking_masks.tif",
    output_path="parking.geojson",
    min_object_area=300,
    # max_object_area=5000,
)

Processing masks from: parking_masks.tif
Found 101 connected components
Using single-threaded processing, you can speed up processing by setting n_workers > 1


Processing components: 100%|██████████| 101/101 [00:00<00:00, 183.01it/s]


Saved 99 objects with confidence to parking.geojson
Total processing time: 1.08 seconds


#8. Add Geometric Properties
Calculate additional attributes like area, perimeter, and bounding box geometry.

In [9]:
gdf = geoai.add_geometric_properties(gdf)

#9. Visualize Detected Parking Spots
Visualize the vectorized detections interactively, colored by confidence score.

In [10]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Final Output: Parking Spot Detection Results

The image shows the **final output** of our parking spot detection pipeline using the `geoai.CarDetector()` model.

- **Overlay**: Yellow polygons indicate **detected vehicles** (cars parked in spots).
- **Purpose**: This visual confirms successful identification of occupied parking spaces by the model.

---

### 🧾 Interpretation Guide

| Element                | Description                                                               |
|------------------------|---------------------------------------------------------------------------|
| 🟨 **Yellow Polygons** | Each represents a detected vehicle (likely occupying a parking space).    |
| 🚗 **Shapes/Sizes**     | Reflects variations in vehicle orientation, size, and image perspective.  |
| ❌ **Empty Spaces**     | Areas without polygons are likely vacant parking spots.                   |
| 🔄 **Model Accuracy**   | Depends on input resolution, chip size, and threshold tuning.             |

---

### ✅ Use Cases of Output

- **🚦 Smart Parking Systems**: Automate real-time occupancy monitoring.
- **🏙️ Urban Planning**: Analyze parking demand and optimize infrastructure.
- **📈 Data Analytics**: Track usage trends across time or geography.
- **🔍 Surveillance & Safety**: Detect anomalies or over-congested zones.



